In [1]:
import nasdaqdatalink as ndl
import pandas as pd
from ta import add_all_ta_features
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np


In [ ]:
#configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data_clean = pd.read_csv('../data/data_clean.csv', parse_dates=['date'])

## Creación de nuevas variables
Modelo Base

In [9]:
data_base = data_clean.copy()

In [10]:
data_base.columns
len(data_base)

12721

En primer lugar, transformaremos ciertas variables fundamentales en variaciones año sobre año. Esto se debe a que el valor absoluto no es tan bueno estimando como el ritmo de crecimiento según la literatura

In [11]:
import numpy as np

# Ordenar primero por ticker y fecha
data_base = data_base.sort_values(['ticker', 'date'])

# 0. Risk Free Rate a Decimal
data_base['risk_free_rate'] = data_base['risk_free_rate'] / 100


# 1. Crecimientos YoY para variables fundamentales
for col in ['revenue', 'ebitda', 'eps', 'equity', 'capex', 'pe', 'pb', 'fcfps', 'ncfo']:
    data_base[f'{col}_yoy'] = data_base.groupby('ticker')[col].pct_change()

# 2. Transformaciones logarítmicas
for col in ['revenue', 'ebitda', 'ebit', 'marketcap', 'equity']:
    data_base[f'log_{col}'] = np.log1p(data_base[col])

# 3. Ratios financieros 
data_base['ebit_margin'] = data_base['ebit'] / data_base['revenue']
data_base['net_margin'] = data_base['netinc'] / data_base['revenue']
data_base['capex_to_revenue'] = data_base['capex'] / data_base['revenue']

# 4. FCF Yield 
data_base['fcf_yield'] = data_base['fcfps'] / data_base['closeadj']

# 5. Price to Sales Ratio
data_base['ps'] = data_base['marketcap'] / data_base['revenue']

# 6. Combinaciones retorno/volatilidad
data_base['ret_12m_vol'] = data_base['ret_12m'] / data_base['volatility_1y']
data_base['ret_6m_vol'] = data_base['ret_6m'] / data_base['volatility_1y']

# 7. Diferencia de medias móviles
data_base['sma_diff'] = data_base['sma_1y'] - data_base['sma_2y']

# 8. Distancia al máximo/mínimo anual
data_base['dist_max_1y'] = data_base['closeadj'] / data_base['max_1y']
data_base['dist_min_1y'] = data_base['closeadj'] / data_base['min_1y']

# 8. Volatilidad relativa
data_base['vol_over_sma'] = data_base['volatility_1y'] / data_base['sma_1y']

# 9. Tamaño por percentil 


p90 = data_base['marketcap'].quantile(0.9)
p70 = data_base['marketcap'].quantile(0.7)
p30 = data_base['marketcap'].quantile(0.3)


def cap_category(marketcap):
    if marketcap >= p90:
        return 'Mega Cap'
    elif marketcap >= p70:
        return 'Large Cap'
    elif marketcap >= p30:
        return 'Mid Cap'
    else:
        return 'Small Cap'

data_base['cap_category'] = data_base['marketcap'].apply(cap_category)
dummies = pd.get_dummies(data_base['cap_category'], prefix='cap')
data_base = pd.concat([data_base, dummies], axis=1)




c:\Users\Ignacio\sp500_env\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Ignacio\sp500_env\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Ignacio\sp500_env\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Ignacio\sp500_env\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [17]:
data_base.columns

Index(['ticker', 'date', 'vol_1y', 'max_1y', 'min_1y', 'sma_1y', 'sma_2y',
       'volatility_1y', 'revenue', 'eps', 'ebitda', 'netinc', 'roe',
       'evebitda', 'fcfps', 'pb', 'pe', 'marketcap', 'ebit', 'capex', 'ncfo',
       'equity', 'de', 'closeadj', 'ret_6m', 'ret_12m', 'risk_free_rate',
       'target_12m_final', 'sharpe_1y', 'revenue_yoy', 'ebitda_yoy', 'eps_yoy',
       'equity_yoy', 'capex_yoy', 'pe_yoy', 'pb_yoy', 'fcfps_yoy', 'ncfo_yoy',
       'log_revenue', 'log_ebitda', 'log_ebit', 'log_marketcap', 'log_equity',
       'ebit_margin', 'net_margin', 'capex_to_revenue', 'fcf_yield', 'ps',
       'ret_12m_vol', 'ret_6m_vol', 'sma_diff', 'dist_max_1y', 'dist_min_1y',
       'vol_over_sma', 'cap_category', 'cap_Large Cap', 'cap_Mega Cap',
       'cap_Mid Cap', 'cap_Small Cap'],
      dtype='object')

In [12]:
data_base.isna().sum()


ticker                 0
date                   0
vol_1y                 0
max_1y                 0
min_1y                 0
sma_1y                 0
sma_2y                 0
volatility_1y          0
revenue                0
eps                    0
ebitda                 0
netinc                 0
roe                    0
evebitda               0
fcfps                  0
pb                     0
pe                     0
marketcap              0
ebit                   0
capex                  0
ncfo                   0
equity                 0
de                     0
closeadj               0
ret_6m                 0
ret_12m                0
risk_free_rate         0
target_12m_final       0
sharpe_1y              0
revenue_yoy         1042
ebitda_yoy          1042
eps_yoy             1043
equity_yoy          1042
capex_yoy           1409
pe_yoy              1042
pb_yoy              1042
fcfps_yoy           1042
ncfo_yoy            1042
log_revenue            4
log_ebitda           454


Al crear variables de crecimiento y logarítmicas optenemos nan's y valores infinitos. El primer aproximamiento va a ser eliminarlas 

### DUDA PROFESOR !?


In [15]:
data_base.dropna()

,ticker,date,vol_1y,max_1y,min_1y,sma_1y,sma_2y,volatility_1y,revenue,eps,...,ret_6m_vol,sma_diff,dist_max_1y,dist_min_1y,vol_over_sma,cap_category,cap_Large Cap,cap_Mega Cap,cap_Mid Cap,cap_Small Cap
1,A,2002-03-31,571608700.0,24.576,11.473,17.982242,26.952406,0.582828,8.396000e+09,0.38,...,1.352362,-8.970164,0.865356,1.853656,0.032411,Mid Cap,False,False,True,False
4,A,2005-03-31,695043500.0,19.807,11.972,14.791570,14.926191,0.352118,7.181000e+09,0.72,...,0.083114,-0.134621,0.681830,1.128049,0.023805,Mid Cap,False,False,True,False
5,A,2006-03-31,773017100.0,23.068,12.410,18.319332,16.562450,0.264804,5.139000e+09,0.66,...,0.553510,1.756882,0.990203,1.840612,0.014455,Mid Cap,False,False,True,False
6,A,2007-03-31,696310300.0,23.816,16.473,20.626060,19.449593,0.283790,4.973000e+09,7.67,...,0.326927,1.176467,0.912454,1.319189,0.013759,Mid Cap,False,False,True,False
7,A,2008-03-31,670314700.0,26.059,18.848,23.120287,21.871240,0.286923,5.420000e+09,1.62,...,-0.666313,1.249047,0.738363,1.020851,0.012410,Mid Cap,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12716,ZTS,2020-03-31,553631975.0,138.935,88.821,115.141528,99.625280,0.342429,6.260000e+09,3.14,...,-0.154178,15.516248,0.811991,1.270128,0.002974,Large Cap,True,False,False,False
12717,ZTS,2021-03-31,468140963.0,167.833,107.283,145.554611,130.348069,0.285314,6.675000e+09,3.44,...,-0.157886,15.206542,0.904631,1.415201,0.001960,Mega Cap,False,True,False,False
12718,ZTS,2022-03-31,459335301.0,239.093,150.303,191.643542,168.644709,0.212392,7.776000e+09,4.29,...,-0.121795,22.998833,0.764669,1.216390,0.001108,Mega Cap,False,True,False,False
12719,ZTS,2023-03-31,524945749.0,193.975,127.862,158.653191,175.213823,0.316182,8.080000e+09,4.51,...,0.402990,-16.560632,0.838541,1.272122,0.001993,Mega Cap,False,True,False,False


In [16]:
len(data_base)

12721

In [18]:
data_base.to_csv('../data/data_base.csv', index=False)